<a href="https://colab.research.google.com/github/NikolasGialitsis/LDA2vec/blob/master/rest_api_topics_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flask-ngrok


In [3]:
!git clone https://github.com/NikolasGialitsis/LDA2vec

Cloning into 'LDA2vec'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 184 (delta 39), reused 0 (delta 0), pack-reused 112
Receiving objects: 100% (184/184), 15.69 MiB | 17.83 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [0]:
# -*- coding: utf-8 -*-
"""LDA2vec_inference.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1szh4AuxOleuFERJPzlqehK-yPRVwka2P
"""


!pip install spacy
!pip install jellyfish
!pip install -r /content/requirements.txt
!pip install pylda2vec




In [0]:
 # Commented out IPython magic to ensure Python compatibility.
%cd /content/LDA2vec/
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

import json

from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

import h5py
%cd LDA2vec/
!python -m spacy download en

In [346]:
def getTopicComposition(query):
  trained = h5py.File('/content/LDA2vec/lda2vec.hdf5', 'r+')
  mixture = trained['mixture']
  for key in mixture.keys():
    print('\t',key)
  factors = mixture.get('factors')['W'][()]
  weights = mixture.get('weights')['W'][()]
  sampler = trained['sampler'].get('W')[()]

  sampler = pickle.load(open('/content/LDA2vec/sampler.pkl', 'rb'))
  factors = pickle.load(open('/content/LDA2vec/factors.pkl', 'rb'))
  vocab = pickle.load(open('/content/LDA2vec/vocab.pkl', 'rb'))
  corpus = pickle.load(open('/content/LDA2vec/corpus.pkl', 'rb'))
  flattened = np.load('/content/LDA2vec/flattened.npy')
  vectors = np.load('/content/LDA2vec/vectors.npy')
  n_vocab = flattened.max() + 1
  print(weights.shape)
  print(factors.shape)
  print(sampler.shape)
  print(flattened[:10])
  titles = []
  abstracts = []
  concats = []
  concat = query.split('------------------------------')  
  title = concat[0]
  abstract = concat[1]
  print('title:',title)
  print('abstract',abstract[:100])
  concats.append(concat[0] + ' ' + concat[1])
  titles.append(title)
  abstracts.append(abstract)

  stop_words = set(stopwords.words('english')) 
  new_text = []
  for text in concats.copy():
    word_tokens = word_tokenize(text) 
    #remove stopwords
    text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
    #remove punctuation
    #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

    text = ''.join([w for w in text if w not in string.punctuation])
    new_text.append(text) 

  max_length = 10000

  new_text = new_text[0].split()
  tokens = np.zeros(max_length)
  print(n_vocab)
  for ident in range(len(new_text)):

    word = new_text[ident]
    if word in vocab.values():
      for k in vocab.keys():
        if word == vocab[k] and k <= n_vocab:
          tokens[ident] = k

  words = corpus.word_list(vocab)[:n_vocab]
  doc_ids = np.zeros(len(tokens))
  doc_ids = np.array([int(d) for d in doc_ids])
  flattened = tokens.copy()
  n_docs = doc_ids.max() + 1
  clambda = 200.0
  # Number of topics to fit
  n_topics = int(os.getenv('n_topics', 10))
  batchsize = 16
  # Power for neg sampling
  power = float(os.getenv('power', 0.75))
  # Intialize with pretrained word vectors
  pretrained = bool(int(os.getenv('pretrained', True)))
  # Sampling temperature
  temperature = float(os.getenv('temperature', 1.0))
  # Number of dimensions in a single word vector
  n_units = int(os.getenv('n_units', 30))
  # Get the string representation for every compact key
  # How many tokens are in each document
  doc_idx, lengths = np.unique(doc_ids, return_counts=True)
  doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
  doc_lengths[doc_idx] = lengths

  flattened = np.array([int(f) for f in flattened])
  tok_idx, freq = np.unique(flattened, return_counts=True)
  term_frequency = np.zeros(n_vocab, dtype='int32')
  term_frequency[tok_idx] = freq

  model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                  n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                  n_samples=15, power=power, temperature=temperature)

  
  model.sampler.W.data[:, :] = vectors[:n_vocab, :]
  npz = np.load(open('/content/LDA2vec/topics.pyldavis.npz', 'rb'))
  dat = {k: v for (k, v) in npz.items()}

  model.to_gpu()
  optimizer = O.Adam()
  optimizer.setup(model)
  clip = chainer.optimizer.GradientClipping(5.0)
  optimizer.add_hook(clip)

  j = 0
  epoch = 0
  fraction = batchsize * 1.0 / flattened.shape[0]
  progress = shelve.open('progress.shelve')
  

  n_epochs = 1
  for epoch in range(n_epochs):
      data = prepare_topics(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(factors).copy(),
                          cuda.to_cpu(sampler).copy(),
                          words)
      data['topic_term_dists'] = dat['topic_term_dists']
      top_words = print_top_words_per_topic(data)
      data['doc_lengths'] = doc_lengths
      data['term_frequency'] = term_frequency
      print(epoch)
      for d, f in utils.chunks(batchsize, doc_ids, flattened):
          t0 = time.time()
          model.cleargrads()
          #optimizer.use_cleargrads(use=False)
          l = model.fit_partial(d.copy(), f.copy(),update_only_docs=True)
          #print("after partial fitting:", l)
          prior = model.prior()
          loss = prior * fraction
          loss.backward()
          optimizer.update()
          msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} ""P:{prior:1.3e} R:{rate:1.3e}")
          prior.to_cpu()
          loss.to_cpu()
          t1 = time.time()
          dt = t1 - t0
          rate = batchsize / dt
          logs = dict(loss=float(l), epoch=epoch, j=j,
                      prior=float(prior.data), rate=rate)
          #print(msg.format(**logs))
          j += 1

  
  top_n = 10
  topic_to_topwords = {}
  for j, topic_to_word in enumerate(data['topic_term_dists']):
      top = np.argsort(topic_to_word)[::-1][:top_n]
      msg = 'Topic %i '  % j
      top_words = [data['vocab'][i].strip()[:35] for i in top]
      msg += ' '.join(top_words)
      print(msg)
      topic_to_topwords[j] = top_words

  ret_val = ""
  t = 0
  for val in data['doc_topic_dists'][-1]:
    ret_val += (str((val*100).round(2)) +'% *'+ str(topic_to_topwords[t][0])+'\n')
    t+=1



  return ret_val 


query = [' assigned  ']*10000
query.append( '------------------------------' )

query = str(query)
print(query)
comp = getTopicComposition(query)
for line in comp.split('\n'):
  print(line)

[' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigned  ', ' assigne

In [251]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import Flask, jsonify
from flask import abort
from flask import make_response
from flask import request

app = Flask(__name__)
run_with_ngrok(app)
tasks = [
    {
        'id': 1,
        'title': u'Buy groceries',
        'description': u'Milk, Cheese, Pizza, Fruit, Tylenol', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn Python',
        'description': u'Need to find a good Python tutorial on the web', 
        'done': False
    }
]


@app.route("/")
def home():
    return "<h1>Infer Topic Compositions on Google Colab!</h1>"

@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

@app.route('/todo/api/v1.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    print(task['title'])
    composition = getTopicComposition(task['title'])
    print(composition) 
    return jsonify({'topic composition': str(composition)}), 201

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4e5be0ec.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
A covid19 paper ------------------------------ Covid19 is a disease caused by the coronavirus SARS-COV2 and scientists from the united states have tried to develop vaccine
	 factors
	 weights
(744, 10)
(10, 30)
(3033, 30)
[  8  17   5  14 487 383  93 447  73   8]
title: A covid19 paper 
abstract  Covid19 is a disease caused by the coronavirus SARS-COV2 and scientists from the united states have
3033
Top words in topic 0 inhibits showed exhibited des containing glycoprotein receptor protein staining induced
Top words in topic 1 inhibits showed exhibited des containing glycoprotein receptor protein induced inhibit
Top words in topic 2 inhibits showed exhibited des containing glycoprotein protein receptor staining sem
Top words in topic 3 inhibits showed exhibited des containing glycoprotein protein receptor staining induced
Top words in topic 4 inhibits showed exhibited containing des glycoprotein 

INFO:werkzeug:127.0.0.1 - - [28/Apr/2020 15:27:20] "POST /todo/api/v1.0/tasks HTTP/1.1" 201 -


Topic 9 inhibits showed exhibited des containing glycoprotein receptor protein staining induced
0.107560255*inhibits0.11502418*inhibits0.12327181*inhibits0.092092715*inhibits0.10287421*inhibits0.059582926*inhibits0.13391791*inhibits0.06787255*inhibits0.13904418*inhibits0.058759224*inhibits
